# European Soccer Events Analysis: ETL
* A soccer game or any other sport generates many events, which help solve a number of use cases across the Sports and Media & Entertainment industries:
  * *Like what on-field playing conditions and events (passes, positions etc.) leads to more goals/touch-downs etc.*
  * *Or what does the win-loss percentage looks like with different combinations of players in different on-field positions*
  * *Or what does a sportsperson's performance graph look like across the years/seasons and teams etc.*

![](https://databricks.com/wp-content/uploads/2018/07/European-Soccer-Events-Analysis-Diagram.png)


* This demo uses a European Soccer Games events dataset, and demonstrates:
  * *End-to-end Data Engineering pipeline including data extraction, transformation and loading*
  * *How to answer business questions by analyzing the transformed data - using a combination of Spark SQL and Visualizations*
  * *Usage of Gradient-boosted tree classifier to predict events of most significance (goals in a soccer game)*

##1. Data Sourcing/Extraction

Dataset has been downloaded from [**Kaggle**](https://www.kaggle.com/secareanualin/football-events). It provides a granular view of 9,074 games, from the biggest 5 European football (soccer) leagues: England, Spain, Germany, Italy, France, from 2011/2012 season to 2016/2017 season as of 25.01.2017. This is what the schema looks like:

| Column Name | Colum Description |
| ----------- | ----------------- |
| id_odsp | unique identifier of game (odsp stands from oddsportal.com) |
| id_event | unique identifier of event (id_odsp + sort_order) |
| sort_order | chronological sequence of events in a game |
| time | minute of the game |
| text | text commentary |
| event_type | primary event, 11 unique events |
| event_type2 | secondary event, 4 unique events |
| side | Home or Away team |
| event_team | team that produced the event. In case of Own goals, event team is the team that benefited from the own goal |
| opponent | opposing team |
| player | name of the player involved in main event |
| player2 | name of player involved in secondary event |
| player_in | player that came in (only applies to substitutions) |
| player_out | player substituted (only applies to substitutions) |
| shot_place | placement of the shot, 13 possible placement locations |
| shot_outcome | 4 possible outcomes |
| is_goal | binary variable if the shot resulted in a goal (own goals included) |
| location | location on the pitch where the event happened, 19 possible locations |
| bodypart | 3 body parts |
| assist_method | in case of an assisted shot, 5 possible assist methods |
| situation | 4 types |

To load your data into DBFS, please refer to [Databricks Guide > Importing Data](https://docs.databricks.com/user-guide/importing-data.html).

In [0]:
dbutils.fs.mkdirs("/data/eu-soccer-events")
dbutils.fs.mkdirs("/data/eu-soccer-events/input")
dbutils.fs.mkdirs("/data/eu-soccer-events/interm")

Out[ 12 ]: True

In [0]:
%fs ls /data/eu-soccer-events/input

In [0]:
%sh head /data/eu-soccer-events/input/events.csv

"id_odsp","id_event","sort_order","time","text","event_type","event_type2","side","event_team","opponent","player","player2","player_in","player_out","shot_place","shot_outcome","is_goal","location","bodypart","assist_method","situation","fast_break"
"UFot0hit/","UFot0hit1",1,2,"Attempt missed. Mladen Petric (Hamburg) left footed shot from the left side of the box is high and wide to the left. Assisted by Gokhan Tore.",1,12,2,"Hamburg SV","Borussia Dortmund","mladen petric","gokhan tore",NA,NA,6,2,0,9,2,1,1,0
"UFot0hit/","UFot0hit2",2,4,"Corner, Borussia Dortmund. Conceded by Dennis Diekmeier.",2,NA,1,"Borussia Dortmund","Hamburg SV","dennis diekmeier","dennis diekmeier",NA,NA,NA,NA,0,NA,NA,0,NA,0
"UFot0hit/","UFot0hit3",3,4,"Corner, Borussia Dortmund. Conceded by Heiko Westermann.",2,NA,1,"Borussia Dortmund","Hamburg SV","heiko westermann","heiko westermann",NA,NA,NA,NA,0,NA,NA,0,NA,0
"UFot0hit/","UFot0hit4",4,7,"Foul by Sven Bender (Borussia Dortmund).",3,NA,1,"Borussia Dortmund","Hamburg SV","sven bender",NA,NA,NA,NA,NA,0,NA,NA,0,NA,0
"UFot0hit/","UFot0hit5",5,7,"Gokhan Tore (Hamburg) wins a free kick in the defensive half.",8,NA,2,"Hamburg SV","Borussia Dortmund","gokhan tore",NA,NA,NA,NA,NA,0,2,NA,0,NA,0
"UFot0hit/","UFot0hit6",6,9,"Hand ball by Jose Paolo Guerrero (Hamburg).",10,NA,2,"Hamburg SV","Borussia Dortmund","jose paolo guerrero",NA,NA,NA,NA,NA,0,NA,NA,0,NA,0
"UFot0hit/","UFot0hit7",7,10,"Corner, Hamburg. Conceded by Lukasz Piszczek.",2,NA,2,"Hamburg SV","Borussia Dortmund","lukasz piszczek","lukasz piszczek",NA,NA,NA,NA,0,NA,NA,0,NA,0
"UFot0hit/","UFot0hit8",8,11,"Chris Lowe (Borussia Dortmund) wins a free kick in the defensive half.",8,NA,1,"Borussia Dortmund","Hamburg SV","chris lowe",NA,NA,NA,NA,NA,0,2,NA,0,NA,0
"UFot0hit/","UFot0hit9",9,11,"Foul by Gojko Kacar (Hamburg).",3,NA,2,"Hamburg SV","Borussia Dortmund","gojko kacar",NA,NA,NA,NA,NA,0,NA,NA,0,NA,0

In [0]:
from pyspark.sql.types import *

schema = (StructType().
          add("id_odsp", StringType()).add("id_event", StringType()).add("sort_order", IntegerType()).
          add("time", IntegerType()).add("text", StringType()).add("event_type", IntegerType()).
          add("event_type2", IntegerType()).add("side", IntegerType()).add("event_team", StringType()).
          add("opponent", StringType()).add("player", StringType()).add("player2", StringType()).
          add("player_in", StringType()).add("player_out", StringType()).add("shot_place", IntegerType()).
          add("shot_outcome", IntegerType()).add("is_goal", IntegerType()).add("location", IntegerType()).
          add("bodypart", IntegerType()).add("assist_method", IntegerType()).add("situation", IntegerType()).
          add("fast_break", IntegerType())
         )

In [0]:
eventsDf = (spark.read.csv("/data/eu-soccer-events/input/events.csv", 
                         schema=schema, header=True, 
                         ignoreLeadingWhiteSpace=True, 
                         ignoreTrailingWhiteSpace=True,
                         nullValue='NA'))

eventsDf = eventsDf.na.fill({'player': 'NA', 'event_team': 'NA', 'opponent': 'NA', 
                             'event_type': 99, 'event_type2': 99, 'shot_place': 99, 
                             'shot_outcome': 99, 'location': 99, 'bodypart': 99, 
                             'assist_method': 99, 'situation': 99})
display(eventsDf)

id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left footed shot from the left side of the box is high and wide to the left. Assisted by Gokhan Tore.,1,12,2,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,null,null,6,2,0,9,2,1,1,0
UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis Diekmeier.",2,99,1,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,null,null,99,99,0,99,99,0,99,0
UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko Westermann.",2,99,1,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,null,null,99,99,0,99,99,0,99,0
UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,99,1,Borussia Dortmund,Hamburg SV,sven bender,null,null,null,99,99,0,99,99,0,99,0
UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the defensive half.,8,99,2,Hamburg SV,Borussia Dortmund,gokhan tore,null,null,null,99,99,0,2,99,0,99,0
UFot0hit/,UFot0hit6,6,9,Hand ball by Jose Paolo Guerrero (Hamburg).,10,99,2,Hamburg SV,Borussia Dortmund,jose paolo guerrero,null,null,null,99,99,0,99,99,0,99,0
UFot0hit/,UFot0hit7,7,10,"Corner, Hamburg. Conceded by Lukasz Piszczek.",2,99,2,Hamburg SV,Borussia Dortmund,lukasz piszczek,lukasz piszczek,null,null,99,99,0,99,99,0,99,0
UFot0hit/,UFot0hit8,8,11,Chris Lowe (Borussia Dortmund) wins a free kick in the defensive half.,8,99,1,Borussia Dortmund,Hamburg SV,chris lowe,null,null,null,99,99,0,2,99,0,99,0
UFot0hit/,UFot0hit9,9,11,Foul by Gojko Kacar (Hamburg).,3,99,2,Hamburg SV,Borussia Dortmund,gojko kacar,null,null,null,99,99,0,99,99,0,99,0
UFot0hit/,UFot0hit10,10,13,Foul by Gokhan Tore (Hamburg).,3,99,2,Hamburg SV,Borussia Dortmund,gokhan tore,null,null,null,99,99,0,99,99,0,99,0


In [0]:
gameInfDf = (spark.read.csv("/data/eu-soccer-events/input/ginf.csv", 
                         inferSchema=True, header=True, 
                         ignoreLeadingWhiteSpace=True, 
                         ignoreTrailingWhiteSpace=True,
                         nullValue="NA"))
display(gameInfDf)

id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-hamburger-UFot0hit/,true,2011-08-05T00:00:00.000+0000,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,null,null,null,null
Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-freiburg-Aw5DflLH/,true,2011-08-06T00:00:00.000+0000,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.6,3.4,null,null,null,null
bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-bremen-kaiserslautern-bkjpaC6n/,true,2011-08-06T00:00:00.000+0000,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.2,4.8,null,null,null,null
CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lorient-CzPV312a/,true,2011-08-06T00:00:00.000+0000,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.5,9.4,null,null,null,null
GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valenciennes-GUOdmtII/,true,2011-08-06T00:00:00.000+0000,F1,2012,france,Caen,Valenciennes,1,0,2.5,3.4,3.45,null,null,null,null
lOpzwMkp/,/soccer/germany/bundesliga-2011-2012/hertha-berlin-nurnberg-lOpzwMkp/,true,2011-08-06T00:00:00.000+0000,D1,2012,germany,Hertha Berlin,Nurnberg,0,1,2.06,3.75,3.95,null,null,null,null
M7PhlM2C/,/soccer/france/ligue-1-2011-2012/brest-evian-tg-M7PhlM2C/,true,2011-08-06T00:00:00.000+0000,F1,2012,france,Brest,Evian Thonon Gaillard,2,2,2.29,3.25,3.85,null,null,null,null
QuWqjrYa/,/soccer/france/ligue-1-2011-2012/ac-ajaccio-toulouse-QuWqjrYa/,true,2011-08-06T00:00:00.000+0000,F1,2012,france,AC Ajaccio,Toulouse,0,2,2.8,3.1,3.05,null,null,null,null
UBZQ4smg/,/soccer/france/ligue-1-2011-2012/nice-lyon-UBZQ4smg/,true,2011-08-06T00:00:00.000+0000,F1,2012,france,Nice,Lyon,1,3,4.5,3.55,2.0,null,null,null,null
Wn69eU5B/,/soccer/germany/bundesliga-2011-2012/koln-wolfsburg-Wn69eU5B/,true,2011-08-06T00:00:00.000+0000,D1,2012,germany,FC Cologne,VfL Wolfsburg,0,3,3.0,3.8,2.54,null,null,null,null


## 2. Data Transformation
Convert the data to a format, such that one could gather meaningful insights from it

In [0]:
def mapKeyToVal(mapping):
    def mapKeyToVal_(col):
        return mapping.get(col)
    return udf(mapKeyToVal_, StringType())

In [0]:
evtTypeMap = {0:'Announcement', 1:'Attempt', 2:'Corner', 3:'Foul', 4:'Yellow card', 5:'Second yellow card', 6:'Red card', 7:'Substitution', 8:'Free kick won', 9:'Offside', 10:'Hand ball', 11:'Penalty conceded', 99:'NA'}

evtTyp2Map = {12:'Key Pass', 13:'Failed through ball', 14:'Sending off', 15:'Own goal', 99:'NA'}

sideMap = {1:'Home', 2:'Away'}

shotPlaceMap = {1:'Bit too high', 2:'Blocked', 3:'Bottom left corner', 4:'Bottom right corner', 5:'Centre of the goal', 6:'High and wide', 7:'Hits the bar', 8:'Misses to the left', 9:'Misses to the right', 10:'Too high', 11:'Top centre of the goal', 12:'Top left corner', 13:'Top right corner', 99:'NA'}

shotOutcomeMap = {1:'On target', 2:'Off target', 3:'Blocked', 4:'Hit the bar', 99:'NA'}

locationMap = {1:'Attacking half', 2:'Defensive half', 3:'Centre of the box', 4:'Left wing', 5:'Right wing', 6:'Difficult angle and long range', 7:'Difficult angle on the left', 8:'Difficult angle on the right', 9:'Left side of the box', 10:'Left side of the six yard box', 11:'Right side of the box', 12:'Right side of the six yard box', 13:'Very close range', 14:'Penalty spot', 15:'Outside the box', 16:'Long range', 17:'More than 35 yards', 18:'More than 40 yards', 19:'Not recorded', 99:'NA'}

bodyPartMap = {1:'Right foot', 2:'Left foot', 3:'Head', 99:'NA'}

assistMethodMap = {0:'None', 1:'Pass', 2:'Cross', 3:'Headed pass', 4:'Through ball', 99:'NA'}

situationMap = {1:'Open play', 2:'Set piece', 3:'Corner', 4:'Free kick', 99:'NA'}

countryCodeMap = {'germany':'DEU', 'france':'FRA', 'england':'GBR', 'spain':'ESP', 'italy':'ITA'}

In [0]:
gameInfDf = gameInfDf.withColumn("country_code", mapKeyToVal(countryCodeMap)("country"))

display(gameInfDf['id_odsp','country','country_code'])

id_odsp,country,country_code
UFot0hit/,germany,DEU
Aw5DflLH/,germany,DEU
bkjpaC6n/,germany,DEU
CzPV312a/,france,FRA
GUOdmtII/,france,FRA
lOpzwMkp/,germany,DEU
M7PhlM2C/,france,FRA
QuWqjrYa/,france,FRA
UBZQ4smg/,france,FRA
Wn69eU5B/,germany,DEU


In [0]:
eventsDf = (
             eventsDf.
             withColumn("event_type_str", mapKeyToVal(evtTypeMap)("event_type")).
             withColumn("event_type2_str", mapKeyToVal(evtTyp2Map)("event_type2")).
             withColumn("side_str", mapKeyToVal(sideMap)("side")).
             withColumn("shot_place_str", mapKeyToVal(shotPlaceMap)("shot_place")).
             withColumn("shot_outcome_str", mapKeyToVal(shotOutcomeMap)("shot_outcome")).
             withColumn("location_str", mapKeyToVal(locationMap)("location")).
             withColumn("bodypart_str", mapKeyToVal(bodyPartMap)("bodypart")).
             withColumn("assist_method_str", mapKeyToVal(assistMethodMap)("assist_method")).
             withColumn("situation_str", mapKeyToVal(situationMap)("situation"))
           )

joinedDf = (
  eventsDf.join(gameInfDf, eventsDf.id_odsp == gameInfDf.id_odsp, 'inner').
  select(eventsDf.id_odsp, eventsDf.id_event, eventsDf.sort_order, eventsDf.time, eventsDf.event_type, eventsDf.event_type_str, eventsDf.event_type2, eventsDf.event_type2_str, eventsDf.side, eventsDf.side_str, eventsDf.event_team, eventsDf.opponent, eventsDf.player, eventsDf.player2, eventsDf.player_in, eventsDf.player_out, eventsDf.shot_place, eventsDf.shot_place_str, eventsDf.shot_outcome, eventsDf.shot_outcome_str, eventsDf.is_goal, eventsDf.location, eventsDf.location_str, eventsDf.bodypart, eventsDf.bodypart_str, eventsDf.assist_method, eventsDf.assist_method_str, eventsDf.situation, eventsDf.situation_str, gameInfDf.country_code)
)

In [0]:
from pyspark.ml.feature import QuantileDiscretizer

joinedDf = QuantileDiscretizer(numBuckets=10, inputCol="time", outputCol="time_bin").fit(joinedDf).transform(joinedDf)

display(joinedDf)

id_odsp,id_event,sort_order,time,event_type,event_type_str,event_type2,event_type2_str,side,side_str,event_team,opponent,player,player2,player_in,player_out,shot_place,shot_place_str,shot_outcome,shot_outcome_str,is_goal,location,location_str,bodypart,bodypart_str,assist_method,assist_method_str,situation,situation_str,country_code,time_bin
UFot0hit/,UFot0hit1,1,2,1,Attempt,12,Key Pass,2,Away,Hamburg SV,Borussia Dortmund,mladen petric,gokhan tore,null,null,6,High and wide,2,Off target,0,9,Left side of the box,2,Left foot,1,Pass,1,Open play,DEU,0.0
UFot0hit/,UFot0hit2,2,4,2,Corner,99,NA,1,Home,Borussia Dortmund,Hamburg SV,dennis diekmeier,dennis diekmeier,null,null,99,NA,99,NA,0,99,NA,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit3,3,4,2,Corner,99,NA,1,Home,Borussia Dortmund,Hamburg SV,heiko westermann,heiko westermann,null,null,99,NA,99,NA,0,99,NA,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit4,4,7,3,Foul,99,NA,1,Home,Borussia Dortmund,Hamburg SV,sven bender,null,null,null,99,NA,99,NA,0,99,NA,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit5,5,7,8,Free kick won,99,NA,2,Away,Hamburg SV,Borussia Dortmund,gokhan tore,null,null,null,99,NA,99,NA,0,2,Defensive half,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit6,6,9,10,Hand ball,99,NA,2,Away,Hamburg SV,Borussia Dortmund,jose paolo guerrero,null,null,null,99,NA,99,NA,0,99,NA,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit7,7,10,2,Corner,99,NA,2,Away,Hamburg SV,Borussia Dortmund,lukasz piszczek,lukasz piszczek,null,null,99,NA,99,NA,0,99,NA,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit8,8,11,8,Free kick won,99,NA,1,Home,Borussia Dortmund,Hamburg SV,chris lowe,null,null,null,99,NA,99,NA,0,2,Defensive half,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit9,9,11,3,Foul,99,NA,2,Away,Hamburg SV,Borussia Dortmund,gojko kacar,null,null,null,99,NA,99,NA,0,99,NA,99,NA,0,None,99,NA,DEU,0.0
UFot0hit/,UFot0hit10,10,13,3,Foul,99,NA,2,Away,Hamburg SV,Borussia Dortmund,gokhan tore,null,null,null,99,NA,99,NA,0,99,NA,99,NA,0,None,99,NA,DEU,1.0


## 3. Data Loading
Load the transformed data to persistent storage, so that it's query-able across notebooks and clusters

In [0]:
%sql 
  CREATE DATABASE IF NOT EXISTS EURO_SOCCER_DB
  LOCATION "dbfs:/FileStore/databricks-abhinav/eu-soccer-events/interm"

In [0]:
%sql USE EURO_SOCCER_DB

In [0]:
joinedDf.write.saveAsTable("GAME_EVENTS", format = "parquet", mode = "overwrite", partitionBy = "COUNTRY_CODE", path = "dbfs:/FileStore/databricks-abhinav/eu-soccer-events/interm/tr-events")

In [0]:
%sql DESCRIBE GAME_EVENTS

col_name,data_type,comment
id_odsp,string,null
id_event,string,null
sort_order,int,null
time,int,null
event_type,int,null
event_type_str,string,null
event_type2,int,null
event_type2_str,string,null
side,int,null
side_str,string,null
